In [1]:
from qft import *

In [2]:
num_q = 5
cir = QFT(num_q)
cir.draw()

                                                                 
q_0: ─■──■──■──■──■─────■─────■─────■────────────────────────────
      │  │  │  │  │     │     │     │                            
q_1: ─■──■──┼──┼──┼──■──┼──■──┼──■──┼──■──■─────■────────────────
            │  │  │  │  │  │  │  │  │  │  │     │                
q_2: ───────■──■──┼──■──┼──■──┼──┼──┼──┼──┼──■──┼──■──■──■───────
                  │     │     │  │  │  │  │  │  │  │  │  │       
q_3: ─────────────■─────■─────┼──■──┼──■──┼──■──┼──■──┼──┼──■──■─
                              │     │     │     │     │  │  │  │ 
q_4: ─────────────────────────■─────■─────■─────■─────■──■──■──■─
                                                                 


In [3]:
print(cir.num_gate)
cir.gate_list

20


[[],
 [[0, 1], [0, 1]],
 [[0, 2], [0, 2]],
 [[0, 3], [1, 2], [0, 3], [1, 2]],
 [[0, 4], [1, 3], [0, 4], [1, 3]],
 [[1, 4], [2, 3], [1, 4], [2, 3]],
 [[2, 4], [2, 4]],
 [[3, 4], [3, 4]]]

In [4]:
import math
def set_parameters(default):
	para = {}
	if default:
		para['T_cz'] = 0.2  #us
		para['T_eff'] = 1.5e6 #us
		para['T_trans'] = 20 # us
		para['AOD_width'] = 3 #um
		para['AOD_height'] = 3 #um
		para['Move_speed'] = 0.55 #um/us
		para['F_cz'] = 0.995

	return para
def compute_fidelity(parallel_gates, all_movements, num_q, gate_num):
	para = set_parameters(True)
	t_total = 0
	t_total += (len(parallel_gates) * para['T_cz']) # cz execution time, parallel
	t_move = 0
	for move in all_movements:
		t_total += (4 * para['T_trans']) # pick/drop/pick/drop
		t_move += (4 * para['T_trans'])
		max_dis = 0
		for each_move in move:
			x1, y1 = each_move[1][0],each_move[1][1]
			x2, y2 = each_move[2][0],each_move[2][1]
			dis = (abs(x2-x1)*para['AOD_width'])**2 + (abs(y2-y1)*para['AOD_height'])**2
			if dis > max_dis:
				max_dis = dis
		max_dis = math.sqrt(max_dis)
		t_total += (max_dis/para['Move_speed'])
		t_move += (max_dis/para['Move_speed'])

	t_idle = num_q * t_total - gate_num * para['T_cz']
	Fidelity = math.exp(-t_idle/para['T_eff']) * (para['F_cz']**gate_num)
	move_fidelity = math.exp(-t_move/para['T_eff'])
	return t_idle, Fidelity, move_fidelity

In [5]:
def get_maps(cir) -> list[list[int]]:
    maps = []
    new_map = [[1,i+1] for i in range(cir.num_qubits)]
    AOD_qubits = []

    def update_AOD_qubits(locations, AOD_qubits,dir=[0,1]):
        new_loc = copy.deepcopy(locations)
        for qubit in AOD_qubits:
            new_loc[qubit][0] += dir[0]
            new_loc[qubit][1] += dir[1]
        return new_loc
            
    maps.append(new_map)
    for i in range(len(cir.gate_list)-1):
        # if i % 2 != 0:
        if i%2 == 0:
            AOD_qubits.append(i//2)
        new_map = update_AOD_qubits(new_map,AOD_qubits)
        # else:
        #     for qubit in range(cir.num_qubits):
        #         new_map[qubit][1] += 1
        maps.append(new_map)
        new_map = copy.deepcopy(maps[-1])
        
    return maps
maps = get_maps(cir)
maps

[[[1, 1], [1, 2], [1, 3], [1, 4], [1, 5]],
 [[1, 2], [1, 2], [1, 3], [1, 4], [1, 5]],
 [[1, 3], [1, 2], [1, 3], [1, 4], [1, 5]],
 [[1, 4], [1, 3], [1, 3], [1, 4], [1, 5]],
 [[1, 5], [1, 4], [1, 3], [1, 4], [1, 5]],
 [[1, 6], [1, 5], [1, 4], [1, 4], [1, 5]],
 [[1, 7], [1, 6], [1, 5], [1, 4], [1, 5]],
 [[1, 8], [1, 7], [1, 6], [1, 5], [1, 5]]]

In [6]:
from Enola.route import *

In [7]:
route = QuantumRouter(num_q,maps,cir.gate_list,[2,2*num_q+1])
route.run()

In [8]:
total_paralled = cir.gate_list
all_movements = []
for num in range(len(maps) - 1):
    for paral_moves in route.movement_list[num]:
        all_movements.append(paral_moves)
gate_num = cir.num_gate
t_idle, Fidelity, move_fidelity = compute_fidelity(total_paralled, all_movements, num_q, gate_num)
print(["Fidelity:", Fidelity])
print(["t_idle:", t_idle])
print(["move_fidelity", move_fidelity])
print(["Movement times", len(all_movements)])
print(["parallel times", len(total_paralled)])

['Fidelity:', 0.9028061313935791]
['t_idle:', 2994.9090909090914]
['move_fidelity', 0.9996012916265293]
['Movement times', 7]
['parallel times', 8]


In [9]:
for n in range(5,16):
    cir = QFT(n)
    maps = get_maps(cir)
    route = QuantumRouter(cir.num_qubits,maps,cir.gate_list,[2,2*num_q+1])
    route.run()
    total_paralled = cir.gate_list
    all_movements = []
    for num in range(len(maps) - 1):
        for paral_moves in route.movement_list[num]:
            all_movements.append(paral_moves)
    gate_num = cir.num_gate
    t_idle, Fidelity, move_fidelity = compute_fidelity(total_paralled, all_movements, num_q, gate_num)
    print(f"QFT_{n}'s Fidelity is: {Fidelity}", )

QFT_5's Fidelity is: 0.9028061313935791
QFT_6's Fidelity is: 0.8581790161406537
QFT_7's Fidelity is: 0.8076209192278608
QFT_8's Fidelity is: 0.7524601511165168
QFT_9's Fidelity is: 0.6940739241259597
QFT_10's Fidelity is: 0.6338320985768271
QFT_11's Fidelity is: 0.5730453683826469
QFT_12's Fidelity is: 0.5129205056546817
QFT_13's Fidelity is: 0.4545246008146743
QFT_14's Fidelity is: 0.3987594641286062
QFT_15's Fidelity is: 0.34634656970135114
